<a href="https://colab.research.google.com/github/dlevonian/california_renewables/blob/master/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# California ISO: renewables output data 2010-2020

## Preprocess and compile a single CSV file

In [0]:
import numpy as np
import time

import os
import re
import csv

import urllib3
urllib3.disable_warnings()

import datetime
import shutil

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path = '/content/drive/My Drive/NYCDSA/california_renewables/'  # Google Drive path

In [0]:
def string_from_date(date):
    year = str(date.year)
    day = str(date.day)
    month = str(date.month)
    if len(day)==1: day='0'+day
    if len(month)==1: month='0'+month
    return year+month+day

In [0]:
date += datetime.timedelta(days=1)

In [0]:
def download_caiso_source(start_date = '4/20/2010', end_date = '4/30/2020'):
    
    """Download the collection of txt files from California ISO website
    All files come in the following format:
    LAST:   http://content.caiso.com/green/renewrpt/20200430_DailyRenewablesWatch.txt
    FIRST:  http://content.caiso.com/green/renewrpt/20100420_DailyRenewablesWatch.txt
    Returns:
        saved txt files in caiso_raw_files/ 
    """
    http = urllib3.PoolManager()
    tic = time.time()

    date = datetime.datetime.strptime(start_date, '%m/%d/%Y')
    counter=0
    while True:
        str_date = string_from_date(date)

        file_name = f'{str_date}_DailyRenewablesWatch.txt'
        url = f'http://content.caiso.com/green/renewrpt/{file_name}'
        
        time.sleep(1+np.random.uniform(0,1))        
        with http.request('GET', url, preload_content=False) as r,\
                    open(path+'caiso_raw_files/'+file_name, 'wb') as out_file:       
            shutil.copyfileobj(r, out_file)
        
        if date==datetime.datetime.strptime(end_date, '%m/%d/%Y'):
            break

        date += datetime.timedelta(days=1)
        
        counter+=1
        if counter%10==0:
            toc = time.time()
            print(f'downloaded and saved {counter} files in {toc-tic:.1f} sec')


In [0]:
def numeric_rows_from_lines(lines, str_date):
    rows = []
    for line in lines:
        numbers = [int(s) for s in re.findall('\d+|-\d+', line)]
        if len(numbers)==8:
            rows.append([str_date]+numbers)  
        elif len(numbers)==7:
            rows.append([str_date]+numbers+[0])
        elif len(numbers)==6:
            rows[numbers[0]-1]+=numbers[1:]
    return rows

In [0]:
def compile_caiso_csv(start_date = '4/20/2010', end_date = '4/30/2020'):

    tic = time.time()
    date = datetime.datetime.strptime(start_date, '%m/%d/%Y')
    date_1 = datetime.datetime.strptime(end_date, '%m/%d/%Y')
    output_file = f'caiso_{string_from_date(date)}_{string_from_date(date_1)}.csv'

    headers = 'DATE HOUR GEOTHERMAL BIOMASS BIOGAS SMALL_HYDRO WIND SOLAR_PV SOLAR_THERMAL RENEWABLES NUCLEAR THERMAL IMPORTS HYDRO'
    if not os.path.exists(path+'output.csv'):
        with open(path+output_file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(headers.split())

    counter=0
    missed_dates=[]

    with open(path+output_file, "a") as f:
        writer = csv.writer(f)

        while True:
            str_date = string_from_date(date)

            file_name = f'{str_date}_DailyRenewablesWatch.txt'
            with open(path+'caiso_raw_files/'+file_name, 'r') as t:
                lines = list(t.readlines())  
            
            try:
                writer.writerows(numeric_rows_from_lines(lines, str_date))
            except:
                missed_dates.append(str_date)            

            counter+=1
            if counter%100==0:
                toc = time.time()
                print(f'compiled {counter} files in {toc-tic:.1f} sec')

            if date==date_1: break
            date += datetime.timedelta(days=1)

    print(f'Compiled all {counter} files.')
    print(missed_dates)

Text files for three dates were corrupt and needed manual data imputation (mean of the neighboring dates):
- 06/30/2011
- 11/06/2011
- 03/12/2017

In [0]:
compile_caiso_csv()

In [7]:
files = os.listdir(path+'caiso_raw_files/')

# files include 3 corrupt originals coupled with imputed clean copies
# so the actual number of files processed by compile_caiso_csv is len(files)-3
len(files)

3667